In [ ]:
from flask import Flask, request, jsonify
import pymysql
import numpy as np
import math

app = Flask(__name__)

# 코사인 유사도 함수 정의
def cos_sim(a, b):
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0  
    return np.dot(a, b) / (norm_a * norm_b)

@app.route('/api/contents/lastProject.ipynb/execute', methods=['POST'])
def execute_code():
    request_data = request.json
    login_id_val = request_data.get('login_id')
    
    # MySQL 연결 설정
    conn = pymysql.connect(host='43.200.6.191', user='rbwls1542',
                           password='a12345', db='test', charset='utf8mb4')
    cursor = conn.cursor()
    cursor.execute('''SELECT * FROM member WHERE login_id = %s''', (login_id_val,))
    rows = cursor.fetchall()

    list_data = [[0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0,0,0], [0,0]]
    important_mapping = {
        '높은 금리': (0, 0),
        '해당 금융권에 대한 평판': (0, 1),
        '우대조건': (0, 2),
    }

    purpose_mapping = {
        '생활비 자금': (1, 0),
        '결혼 자금': (1, 1),
        '여행 자금': (1, 2),
        '투자 자금': (1, 3),
        '취미활동을 위한 자금': (1, 4),
        '원하는 물건 구매 자금': (1, 5),
    }

    status_mapping = {
        '환경': (2, 0),
        '직장인': (2, 1),
        '청년': (2, 2),
        '퇴직': (2, 3),
        '스포츠': (2, 4),
        '사회적 약자': (2, 5),
        '군인': (2, 6),
        '가족': (2, 7),
        '대학생': (2, 8),
        '학생': (2, 9),
    }
    
    age_mapping = {
        '20대': (3, 0),
        '30대': (3, 1),
        '40대이상': (3, 2)
    }

    gender_mapping = {
        '남성': (4, 0),
        '여성': (4, 1)  
    }
    

    # 사용자 데이터 매핑
    for i in rows:
        my_list = list(i)
        for key, (row_idx, col_idx) in important_mapping.items():
            if key in my_list[2]:
                list_data[row_idx][col_idx] = 1

        for key, (row_idx, col_idx) in purpose_mapping.items():
            if key in my_list[7]:
                list_data[row_idx][col_idx] = 1

        for key, (row_idx, col_idx) in status_mapping.items():
            if key in my_list[8]:
                list_data[row_idx][col_idx] = 1
                
        for key, (row_idx, col_idx) in status_mapping.items():
            if key in my_list[10]:
                list_data[row_idx][col_idx] = 1
                
        for key, (row_idx, col_idx) in status_mapping.items():
            if key in my_list[11]:
                list_data[row_idx][col_idx] = 1
            

    cursor.execute('''SELECT * FROM member WHERE login_id != %s''', (login_id_val,))
    member_rows = cursor.fetchall()

    test_data = [[[0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[0,0,0], [0,0]] for _ in range(len(member_rows))]
    a = 0

    for i in member_rows:
        test_list = list(i)
        for key, (row_idx, col_idx) in important_mapping.items():
            if key in test_list[2]:
                test_data[a][row_idx][col_idx] = 1

        for key, (row_idx, col_idx) in purpose_mapping.items():
            if key in test_list[7]:
                test_data[a][row_idx][col_idx] = 1

        for key, (row_idx, col_idx) in status_mapping.items():
            if key in test_list[8]:
                test_data[a][row_idx][col_idx] = 1
                
        for key, (row_idx, col_idx) in status_mapping.items():
            if key in test_list[10]:
                test_data[a][row_idx][col_idx] = 1

        for key, (row_idx, col_idx) in status_mapping.items():
            if key in test_list[11]:
                test_data[a][row_idx][col_idx] = 1
        a += 1

    cos_storage_data = [[0, 0, 0, 0, 0] for _ in range(len(member_rows))]
    for i in range(len(test_data)):
        for j in range(len(test_data[i])):
            similarity_data = cos_sim(list_data[j], test_data[i][j])
            cos_storage_data[i][j] = similarity_data

    uclide_storage_data = [[0, 0] for _ in range(len(member_rows))]
    for i in range(len(test_data)):
        uclide_data_one = math.sqrt((int(my_list[1]) - int(member_rows[i][1])) ** 2)
        uclide_data_two = math.sqrt((int(my_list[5]) - int(member_rows[i][5])) ** 2)

        uclide_storage_data[i][0] = 1 / (uclide_data_one + 1)
        uclide_storage_data[i][1] = 1 / (uclide_data_two + 1)
 
    current_max = 0
    index = 0
    for i in range(len(cos_storage_data)):
        max_data = sum(cos_storage_data[i])/5 + sum(uclide_storage_data[i])/2
        if current_max < max_data:
            current_max = max_data
            index = i

    similar_member_id = member_rows[index][3]

    cursor.execute('''SELECT member_id FROM member WHERE login_id = %s''', (similar_member_id,))
    result = cursor.fetchone()

    cursor.execute('''SELECT 적금id FROM favorite_install WHERE member_id = %s''', (result[0],))
    적금_ids = cursor.fetchall()

    cursor.execute('''SELECT 예금id FROM favorite_deposit WHERE member_id = %s''', (result[0],))
    예금_ids = cursor.fetchall()

    install_data = []
    for 적금_id in 적금_ids:
        cursor.execute('''SELECT 금융회사명,상품명,가입기간,가입금액,우대조건,금리 FROM installment WHERE 적금id = %s''', (적금_id[0],))
        result = cursor.fetchone()
        if result:
            install_data.append({
                "금융회사명": result[0],
                "상품명": result[1],
                "가입기간": result[2],
                "가입금액": result[3],
                "우대조건": result[4],
                "금리": result[5]
            })

    deposit_data = []
    for 예금_id in 예금_ids:
        cursor.execute('''SELECT 금융회사명,상품명,가입기간,가입금액,우대조건,금리 FROM deposit WHERE 예금id = %s''', (예금_id[0],))
        result = cursor.fetchone()
        if result:
            deposit_data.append({
                "금융회사명": result[0],
                "상품명": result[1],
                "가입기간": result[2],
                "가입금액": result[3],
                "우대조건": result[4],
                "금리": result[5]
            })

    cursor.close()
    conn.close()

    response = {
        "install_data": install_data,
        "deposit_data": deposit_data
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run(port=8888)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8888
Press CTRL+C to quit
